In [ ]:
from collections import Counter
import re

# Path to the text file
txt_path = '../O2_Pipeline/oracionesIskonawa.txt'

# Read the text file
with open(txt_path, 'r', encoding='utf-8') as f:
    text = f.read()

# Tokenize the text (this example uses a simple whitespace tokenizer)
tokens = re.findall(r'\S+', text)

# Count the frequency of each token
token_counts = Counter(tokens)

# Write the tokens and their counts to vocab.txt
with open('vocab.txt', 'w', encoding='utf-8') as f:
    for token, count in token_counts.items():
        f.write(f"{token}\n{count}\n")

print("vocab.txt has been created successfully.")

In [ ]:
#Prepara data 
import subprocess

# Define the paths to the vocabulary and text files
voc_path = 'vocab.txt'
txt_path = 'train.txt'

# Construct the command
command = ['python', 'preprocess/preprocess.py', voc_path, txt_path]

# Run the command in a try-except block
try:
    result = subprocess.run(command, check=True, capture_output=True, text=True)
    print("Command executed successfully")
    print("Output:\n", result.stdout)
    print("Error (if any):\n", result.stderr)
except subprocess.CalledProcessError as e:
    print("An error occurred while executing the command")
    print("Return code:", e.returncode)
    print("Output:\n", e.stdout)
    print("Error:\n", e.stderr)

In [ ]:
#Modelo https://dl.fbaipublicfiles.com/XLM/mlm_tlm_xnli15_1024.pth
#BPE https://dl.fbaipublicfiles.com/XLM/codes_xnli_15
#Vocab https://dl.fbaipublicfiles.com/XLM/vocab_xnli_15

#Descargar en XNLG/base 
!wget https://dl.fbaipublicfiles.com/XLM/mlm_tlm_xnli15_1024.pth
!wget https://dl.fbaipublicfiles.com/XLM/codes_xnli_15
!wget https://dl.fbaipublicfiles.com/XLM/vocab_xnli_15

In [ ]:
import subprocess

# Define the command as a list of arguments
command = [
    "python", "-m", "torch.distributed.launch", "--nproc_per_node=4", "xnlg-train.py",
    "--exp_name", "stage1_en-zh-fr",
    "--dump_path", "./dump",
    "--data_path", "./data/processed/XNLG",
    "--lgs", "en-fr-zh",
    "--mlm_steps", "en,zh,fr,en-fr,en-zh",
    "--emb_dim", "1024",
    "--n_layers", "12",
    "--n_heads", "16",
    "--dropout", "0.1",
    "--attention_dropout", "0.1",
    "--gelu_activation", "true",
    "--batch_size", "32",
    "--bptt", "256",
    "--optimizer", "adam,lr=0.0001",
    "--epoch_size", "300000",
    "--max_epoch", "100000",
    "--validation_metrics", "_valid_mlm_ppl",
    "--stopping_criterion", "_valid_mlm_ppl,25",
    "--fp16", "true"
]

# Run the command in a try-except block
try:
    result = subprocess.run(command, check=True, capture_output=True, text=True)
    print("Command executed successfully")
    print("Output:\n", result.stdout)
    print("Error (if any):\n", result.stderr)
except subprocess.CalledProcessError as e:
    print("An error occurred while executing the command")
    print("Return code:", e.returncode)
    print("Output:\n", e.stdout)
    print("Error:\n", e.stderr)

In [ ]:
#Descargas etapa 2
#Modelo https://drive.google.com/uc?export=download&confirm=1bsX&id=1LKXlEHsCOpt1NOEpjXy1XbXqVk3gZGZQ
#BPE https://drive.google.com/uc?authuser=0&id=1nEIWJQlLD26vPt_22pDnEZSJ4mIjssps&export=download
#Vocab https://drive.google.com/uc?id=1lIERR1ejW7_LV2rL9fFNkEhzmr7jooRx&export=download



In [ ]:
import subprocess

# Define the command as a list of arguments
command = [
    "python", "-m", "torch.distributed.launch", "--nproc_per_node=4", "xnlg-train.py",
    "--exp_name", "stage2_en-zh-fr",
    "--dump_path", "./dump",
    "--data_path", "./data/processed/XNLG",
    "--lgs", "ar-bg-de-el-en-es-fr-hi-ru-sw-th-tr-ur-vi-zh",
    "--mt_steps", "en-zh,zh-en,en-fr,fr-en",
    "--ae_steps", "en,zh,fr",
    "--reload_model", "/path/to/mlm_tlm_xnli15_1024.pth,/path/to/mlm_tlm_xnli15_1024.pth",
    "--emb_dim", "1024",
    "--n_layers", "6",
    "--n_heads", "8",
    "--dropout", "0.1",
    "--attention_dropout", "0.1",
    "--gelu_activation", "True",
    "--batch_size", "16",
    "--bptt", "256",
    "--optimizer", "adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001",
    "--epoch_size", "10000",
    "--max_vocab", "95000",
    "--encoder_only", "False",
    "--train_model_names", "decoder",
    "--stopping_criterion", "valid_en-zh_mt_bleu,25",
    "--validation_metrics", "valid_en-zh_mt_bleu,valid_en-fr_mt_bleu",
    "--eval_bleu", "True",
    "--word_shuffle", "3",
    "--word_dropout", "0.1",
    "--word_blank", "0.1",
    "--lambda_ae", "0.5",
    "--n_enc_layers", "10"
]

# Run the command in a try-except block
try:
    result = subprocess.run(command, check=True, capture_output=True, text=True)
    print("Command executed successfully")
    print("Output:\n", result.stdout)
    print("Error (if any):\n", result.stderr)
except subprocess.CalledProcessError as e:
    print("An error occurred while executing the command")
    print("Return code:", e.returncode)
    print("Output:\n", e.stdout)
    print("Error:\n", e.stderr)